In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

In [ ]:
compustat = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/Nøgetal/Compustat_permincl.csv.zip")

In [ ]:
import numpy as np

In [ ]:
#getting na columns funktion
def get_na_columns (df):
  listings_na_temp = df.isna().sum().sort_values(ascending=False)
  listings_na_temp = listings_na_temp[listings_na_temp > 0]
  return listings_na_temp



In [ ]:
get_na_columns(compustat)

In [ ]:
dict_columns ={"apq":	"Accounts_payable(Apq)",
"atq":	"Assets_total(Atq)",
"ceqq":	"Common_equity(Ceqq)",
"cogsq":	"COGS(Cogsq)",
"dpq":	"Deprect_and_ammor(Dpq)",
"gdwlq": 	"Goodwill_net(Gdwlq)",
"nopiq":	"Non_operating_income(Nopiq)",
"oiadpq":	"Operating_income_after_deprec(Oiadpq)",
"piq":	"Pretax_income(Piq)",
"ppentq":	"PP&E_total_net(Ppentq)",
"rectq":	"Receviables(Rectq)",
"revtq":	"Revenue_total(Revtq)",
"txtq":	"Income taxes_total(Txtq)",
"xoprq":	"Opex_total(Xoprq)",
"xsgaq":	"SG&A(XSGAQ)",
"rdq": "Report_Date_Earnings(Rdq)",
"datadate": "date",
"fyearq": "FY", 
"fyr": "FY_end",
"tic": "ticker",
"conm": "company_name",
"curcdq": "currency",
"curncdq": "native_currency",
"datacqtr": "data_year_quarter",
"datafqtr": "Fiscal_year_quarter",
"rp": "reporting_frq",
"rcpq": "Restructuring_Cost_Pretax(rcpq)",
"xrdq": "R&D(xrdq)",
"costat": "company_status",
"dvpspq": "DPS_pay_date(dvpspq)",
"dvpsxq": "DPS_ex_date(dvpsxq)",
"mkvaltq": "market_value(mkvaltq)",
"loc": "HQ_country_code(loc)",
"sic": "industry_code(sic)",

}


In [ ]:
compustat.rename(columns=dict_columns, inplace=True)

In [ ]:
get_na_columns(compustat)

#Data cleaning

In [ ]:
#dropper kolonner med mange NA værdier (SG&A viste sig at have mange NA i det cleanede data)
compustat.drop(["Restructuring_Cost_Pretax(rcpq)",  ], axis = "columns", inplace=True)

In [ ]:
compustat= compustat[compustat["industry_code(sic)"].notna()]
                    
                   

In [ ]:
compustat.head()

In [ ]:
actuals = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/Acutal data/actual_longrun_falseindex.csv")

In [ ]:
actuals.info()

In [ ]:
get_na_columns(actuals)

In [ ]:
actuals = actuals[actuals["Earnings Per Share Reported - Actual"].notna()]

In [ ]:
get_na_columns(actuals)

In [ ]:
# Ikke behov for den - Beregner selv 1 month total return fra CRSP data
actuals.drop(["Selling General & Administrative Expenses - Total","1 Month Total Return"], axis=1, inplace=True)

In [ ]:
actuals

In [ ]:
# Tjekker for Apple aktien
actuals[actuals.Instrument=="037833100"]

In [ ]:
compustat["date"] = pd.to_datetime(compustat.date)

In [ ]:
actuals["Quarter_end_date"] = pd.to_datetime(actuals["Period End Date"])

In [ ]:
actuals

In [ ]:
# Tjekke om Eikon har duplikater

actuals[actuals.duplicated(subset=["Instrument","Quarter_end_date"])==False]

In [ ]:
# Fjerner alle duplikater der har samme cusip og quarter end date
actuals = actuals[actuals.duplicated(subset=["Instrument","Quarter_end_date"])==False]

In [ ]:
actuals.to_csv("/content/drive/MyDrive/Cand.fælles/Speciale/actuals_for_2Q_est.csv")

In [ ]:
actuals_merged = actuals.merge(compustat, how="inner", left_on=["Quarter_end_date", "Instrument"], right_on=["date", "cusip"])

In [ ]:
actuals_merged.info()

In [ ]:
print(compustat.GVKEY.nunique())
print(compustat.cusip.nunique())

In [ ]:
get_na_columns(actuals_merged)

In [ ]:
compustat[compustat.GVKEY ==180923]

In [ ]:
# Kvartaler skal slutte i marts, juni, september eller december
actuals_merged = actuals_merged[(actuals_merged.FY_end == 3)|(actuals_merged.FY_end == 6)|(actuals_merged.FY_end == 9)|(actuals_merged.FY_end == 12)]

In [ ]:
get_na_columns(actuals_merged)

In [ ]:
actuals_merged.info()

In [ ]:
# Beholder kun exdate
actuals_merged.drop("DPS_pay_date(dvpspq)", axis=1, inplace=True)

In [ ]:
actuals_merged.drop("R&D(xrdq)",axis=1,inplace=True)

In [ ]:
get_na_columns(actuals_merged)

In [ ]:
actuals_merged[actuals_merged["Revenue_total(Revtq)"].isna()]

In [ ]:
import numpy as np

In [ ]:
i = -1
for column in actuals_merged.columns:
  i+=1
  print(f'{i}: {column}')


In [ ]:
def fama (x):
  if 100 <= x <=999 or 2000 <= x <=2399 or 2700 <= x <=2749 or 2770 <= x <=2799 or 3100 <= x <=3199 or 3940 <= x <=3989 :
    return ("Consumer Nondurables")

  elif 2500 <= x <=2519 or 2590 <= x <=2599 or 3630 <= x <=3659 or 3710 <= x <=3711 or 3714 <= x <=3714 or 3716 <= x <=3716 or 3750 <= x <=3751 or 3792 <= x <=3792 or 3900 <= x <=3939 or 3990 <= x <=3999 :
    return ("Consumer durables")

  elif 2520 <= x <=2589 or 2600 <= x <=2699 or 2750 <= x <=2769 or 3000 <= x <=3099 or 3200 <= x <=3569 or 3580 <= x <=3629 or 3700 <= x <=3709 or 3712 <= x <=3713 or 3715 <= x <=3715 or 3717 <= x <=3749 or 3752 <= x <=3791 or 3793 <= x <=3799 or 3830 <= x <=3839 or 3860 <= x <=3899  :
    return ("Manufacturing")

  elif 1200 <= x <=1399 or 2900 <= x <=2999 :
    return ("Energy")

  elif 2800 <= x <=2829 or 2840 <= x <=2899 :
    return ("Chemicals and Allied products")

  elif 3570 <= x <=3579 or 3660 <= x <=3692 or 3694 <= x <=3699 or 3810 <= x <=3829 or 7370 <= x <=7379 :
    return ("Business Equipment")

  elif 4800 <= x <=4899 :
    return ("telecom")

  elif 4900 <= x <=4949 :
    return ("Utilities")

  elif 5000 <= x <=5999 or 7200 <= x <=7299 or 7600 <= x <=7699 :
    return ("Wholesale/Retail")

  elif 2830 <= x <=2839 or 3693 <= x <=3693 or 3840 <= x <=3859 or 8000 <= x <=8099  :
    return ("Healthcare")

  elif 6000 <= x <=6999 :
    return ("finance")

  else: 
    return ("other")

  



In [ ]:
actuals_merged["industry_fama"] = actuals_merged["industry_code(sic)"].apply(lambda x: fama(x=x))

In [ ]:
actuals_merged.industry_fama.value_counts(normalize=True)

In [ ]:
actuals_merged

In [ ]:
get_na_columns(actuals_merged[actuals_merged.industry_fama != "finance"])

In [ ]:
get_na_columns(actuals_merged)

In [ ]:
get_na_columns(actuals_merged[actuals_merged.industry_fama == "finance"])

In [ ]:
i = -1
for column in actuals_merged.columns:
  i+=1
  print(f'{i}: {column}')


#Merging af aktiedata og actuals EPS data

In [ ]:
aktie_data = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/aktie_data_for_merging.csv")

In [ ]:
aktie_data.info()

In [ ]:
aktie_data = aktie_data[["date", "PERMNO", "CUSIP", "VOL", "SHROUT", "adj_close", "CFACSHR", "PRC", "adj_return"]]

In [ ]:
from pandas.tseries.offsets import MonthEnd
aktie_data['EndOfMonth'] = pd.to_datetime(aktie_data['date'])+ MonthEnd(0)

In [ ]:
aktie_data.head()

In [ ]:
aktie_data.info()

In [ ]:
aktie_data


In [ ]:
actuals_merged.info()

In [ ]:
actuals_merged

In [ ]:
actuals_merged[actuals_merged.ticker=="AAPL"]

In [ ]:
actuals_merged["Calc Date"] = pd.to_datetime(actuals_merged["Calc Date"])

In [ ]:
aktie_data.info()

In [ ]:
actuals_merge_aktie = actuals_merged.merge(aktie_data, how="inner", left_on=["Calc Date","perm"], right_on=["EndOfMonth","PERMNO"])

In [ ]:
actuals_merged.info()

In [ ]:
get_na_columns(actuals_merge_aktie)

#Cleaning of Actual_merge_aktie

In [ ]:
!pip install pyjanitor
import janitor

In [ ]:
actuals_merge_aktie = actuals_merge_aktie.groupby_agg(by="GVKEY", agg="count", agg_column_name="GVKEY", new_column_name="count_obs")

In [ ]:
get_na_columns(actuals_merge_aktie)

In [ ]:
actuals_merge_aktie = actuals_merge_aktie[actuals_merge_aktie["adj_return"].notna()]

In [ ]:
get_na_columns(actuals_merge_aktie)

In [ ]:
actuals_merge_aktie['HQ_country_code(loc)'].value_counts()

In [ ]:
actuals_merge_aktie = actuals_merge_aktie[actuals_merge_aktie['HQ_country_code(loc)']=="USA"]

In [ ]:
actuals_merge_aktie.native_currency.value_counts()


In [ ]:
actuals_merge_aktie = actuals_merge_aktie[actuals_merge_aktie['native_currency']=="USD"]

In [ ]:
actuals_merge_aktie.query("count_obs >11", inplace=True)

In [ ]:
actuals_merge_aktie.info()

In [ ]:
get_na_columns(actuals_merge_aktie)

In [ ]:
actuals_merge_aktie = actuals_merge_aktie[actuals_merge_aktie.adj_return!=np.inf]

In [ ]:
get_na_columns(actuals_merge_aktie)

In [ ]:
actuals_merge_aktie

In [ ]:
actuals_merge_aktie["market_cap"] = actuals_merge_aktie["SHROUT"]*actuals_merge_aktie["adj_close"]*actuals_merge_aktie["CFACSHR"]

In [ ]:
get_na_columns(actuals_merge_aktie)

In [ ]:
actuals_merge_aktie[actuals_merge_aktie.ticker=="AAPL"]

## Analytiker data

In [ ]:
analytiker_data = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/Analyst data/analyst_Q1_longrun_falseindex.csv")

In [ ]:
analytiker_data

In [ ]:
get_na_columns(analytiker_data)

In [ ]:
analytiker_data.dropna(subset=["Earnings Per Share Reported - Mean"], inplace=True)

In [ ]:
analytiker_data

In [ ]:
analytiker_data.rename(columns={"Earnings Per Share Reported - Mean": "analyst_EPS_mean", "Earnings Per Share Reported - Standard Deviation": "analyst_std_mean", "Earnings Per Share Reported - High": "analyst_high", "Earnings Per Share Reported - Low": "analyst_low" }, inplace=True)

In [ ]:
analytiker_data

In [ ]:
analytiker_data["Calc Date"] = pd.to_datetime(analytiker_data["Calc Date"])
analytiker_data["Period End Date"] = pd.to_datetime(analytiker_data["Period End Date"])

In [ ]:
analytiker_data.info()

In [ ]:
# Tjekker for duplikater igen
analytiker_data[analytiker_data.duplicated(subset=["Period End Date","Instrument"])]

In [ ]:
analytiker_data = analytiker_data[analytiker_data.duplicated(subset=["Period End Date","Instrument"])==False]

In [ ]:
data_final = (actuals_merge_aktie.merge(analytiker_data, how="left", left_on=["Instrument", "Calc Date"], right_on=["Instrument", "Calc Date"]))

In [ ]:
data_final.drop("market_value(mkvaltq)", axis=1, inplace=True)

In [ ]:
get_na_columns(data_final)

In [ ]:
data_final.info()

In [ ]:
i = -1
for column in data_final.columns:
  i+=1
  print(f'{i}: {column}')


In [ ]:
data_test = data_final.groupby("Instrument").apply(lambda x: x.notnull().mean())

In [ ]:
data_test.rename(columns={"Instrument": "ins"},inplace=True )
data_test.reset_index()

In [ ]:
data_test.describe()

`


In [ ]:
list_na_drop = data_test[(data_test["Accounts_payable(Apq)"]==0) | (data_test["Common_equity(Ceqq)"]==0) |(data_test["Deprect_and_ammor(Dpq)"]==0) | (data_test["Goodwill_net(Gdwlq)"]==0) | (data_test["Non_operating_income(Nopiq)"]==0) | (data_test["Operating_income_after_deprec(Oiadpq)"]==0) | (data_test["SG&A(XSGAQ)"]==0) | (data_test["Revenue_total(Revtq)"]==0) | (data_test["PP&E_total_net(Ppentq)"]==0) | (data_test["Receviables(Rectq)"]==0)].index.to_list()

In [ ]:
data_final = data_final[~data_final.Instrument.isin(list_na_drop)]

In [ ]:
data_imputing = data_final.iloc[:,np.r_[0,3, 23:41]]


In [ ]:
data_imputing.shape

In [ ]:
import missingno as msno

In [ ]:
msno.heatmap(data_imputing)

In [ ]:
data_imputing.isnull().sum(axis=0)

`


In [ ]:
get_na_columns(data_imputing)

In [ ]:
data_imputing[data_imputing["Receviables(Rectq)"].isna()].Instrument.unique()

In [ ]:
data_imputing.to_csv("/content/drive/MyDrive/Cand.fælles/Speciale/R_input.csv")

#IMPUTING AF DATA ved brug af Pandas og autoimputer

In [ ]:
data_imputing

In [ ]:
data_imputing["Period End Date_x"] = pd.to_datetime(data_imputing["Period End Date_x"])

In [ ]:
data_imputing.set_index("Period End Date_x", inplace=True)

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

In [ ]:
list_impute= data_imputing.Instrument.unique().tolist()

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

impute_250 = list(chunks(list_impute,250))


In [ ]:
df_impute_final =[]

for li in impute_250:
    df_temp = data_imputing[data_imputing.Instrument.isin(li)].groupby('Instrument').apply(lambda x: x.interpolate(method='time', limit_area ="inside", limit=2))
    df_impute_final.append(df_temp)
    print("Success")


In [ ]:
get_na_columns(data_imputing)

In [ ]:
data_imputing.info()

In [ ]:
df_impute_final = pd.concat(df_impute_final)

In [ ]:
df_impute_final.info()

In [ ]:
get_na_columns(df_impute_final)

In [ ]:
df_impute_final.info()

In [ ]:
get_na_columns(df_impute_final)

### Test af impute resultater

In [ ]:
instrument_choice="693366205"
data_graph = df_impute_final[df_impute_final.Instrument==instrument_choice]
data_graph_before = data_imputing[data_imputing.Instrument==instrument_choice]

In [ ]:
data_graph.info()

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
test = data_graph_before["Receviables(Rectq)"].to_frame().reset_index()
test_1 = data_graph["Receviables(Rectq)"].to_frame().reset_index()

In [ ]:
test.plot.scatter(x="Period End Date_x", y="Receviables(Rectq)")


In [ ]:
test_1.plot.scatter(x="Period End Date_x", y="Receviables(Rectq)")

In [ ]:
data_graph["Receviables(Rectq)"].plot()

In [ ]:
data_final

In [ ]:
df_impute_final.info()

#Merging imputed data with main data

In [ ]:
data_final.info()

In [ ]:
df_impute_final.info()

In [ ]:
data_final.info()

In [ ]:
data_final.reset_index(inplace=True)
df_impute_final.reset_index(inplace=True)

In [ ]:
data_final.sort_values(by=["Instrument","Period End Date_x"],inplace=True)
df_impute_final.sort_values(by=["Instrument","Period End Date_x"],inplace=True)

In [ ]:
data_final.iloc[13450,:]

In [ ]:
df_impute_final.iloc[130450,:]

In [ ]:
get_na_columns(df_impute_final)

In [ ]:
get_na_columns(data_final)

In [ ]:
data_final.info()

In [ ]:
df_impute_final.set_index(["Instrument","Period End Date_x"],inplace=True)
data_final.set_index(["Instrument","Quarter_end_date"],inplace=True)

In [ ]:
combine_df =data_final.combine_first(df_impute_final)

In [ ]:
get_na_columns(combine_df)

In [ ]:
combine_df.reset_index(inplace=True)

In [ ]:
combine_df

In [ ]:
combine_df["SG&A(XSGAQ)"].value_counts()

# Sammenligne NA før og efter imputing

In [ ]:
get_na_columns(data_imputing)

In [ ]:
df_compare = get_na_columns(data_final).to_frame()

In [ ]:
df_compare["efter_imputing"] = get_na_columns(df_impute_final).to_frame()

In [ ]:
df_compare.to_excel("Sammenligning.xlsx")

In [ ]:
get_na_columns(df_impute_final)

In [ ]:
get_na_columns(combine_df).index.to_list()

In [ ]:
combine_df.drop("Goodwill_net(Gdwlq)",axis=1,inplace=True)

In [ ]:
subsetna_list = ['Deprect_and_ammor(Dpq)',
 'Receviables(Rectq)',
 'PP&E_total_net(Ppentq)',
 'Accounts_payable(Apq)',
 'Income taxes_total(Txtq)',
 'Common_equity(Ceqq)',
 'Operating_income_after_deprec(Oiadpq)',
 'Opex_total(Xoprq)',
 'COGS(Cogsq)',
 'Non_operating_income(Nopiq)',
 'Pretax_income(Piq)',
 'Revenue_total(Revtq)',
 'niq',]

In [ ]:
get_na_columns(combine_df)

In [ ]:
combine_df.dropna(subset=subsetna_list,inplace=True)

In [ ]:
get_na_columns(combine_df)

In [ ]:
get_na_columns(df_impute_final)

In [ ]:
combine_df.info()

In [ ]:
combine_df.industry_fama.value_counts(normalize=True)

In [ ]:
combine_df.shape

# Scaling data

In [ ]:
scaled_df = combine_df.copy()

In [ ]:
def scaling_function(x,atq):
  result = x/(max(1,atq))
  return result

In [ ]:
scaled_df[scaled_df["Assets_total(Atq)"]<10]

In [ ]:
scaling_function(500000000,0)

In [ ]:
scaled_df.info()

In [ ]:
scaled_df.iloc[:,np.r_[2:4,5,8,10,18:22,23,27:29,30:32,57]]

In [ ]:
scaled_df.iloc[:,np.r_[2:4,5,8,10,18:22,23,27:29,30:32,57]] =scaled_df.iloc[:,np.r_[2:4,5,8:11,18:22,23,27:29,30:32,57]].apply(lambda column: scaling_function(column,column["Assets_total(Atq)"]),axis=1)

In [ ]:
scaled_df[scaled_df.Instrument =="037833100"]

In [ ]:
scaled_df.info()

# Adding Fama french 

In [ ]:
fama_return = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/Fama French/returns_industry.csv",sep=";")

In [ ]:
fama_return.rename(columns={"Unnamed: 0": "date"},inplace=True)

In [ ]:
fama_return.head()

In [ ]:
fama_return["date_format"] = pd.to_datetime(fama_return["date"],format="%Y%m")+ MonthEnd(0)

In [ ]:
fama_return.drop("date",axis=1,inplace=True)

In [ ]:
column_rename_dict = {'Manuf':'Manufacturing',
 'NoDur':'Consumer Nondurables',
 'Shops':'Wholesale/Retail',
 'BusEq':'Business Equipment',
 'Hlth ':'Healthcare',
 'Other':'other',
 'Telcm':'telecom',
 'Chems':'Chemicals and Allied products',
 'Enrgy':'Energy',
 'Money':'finance',
 'Durbl':'Consumer durables',
 'Utils':'Utilities'}

In [ ]:
fama_return.rename(columns=column_rename_dict,inplace=True)

In [ ]:
fama_return

In [ ]:
fama_return_long = pd.melt(fama_return,id_vars="date_format")

In [ ]:
scaled_df.info()

In [ ]:
scaled_df_fama = scaled_df.merge(fama_return_long,how="left",left_on=["Calc Date","industry_fama"],right_on=["date_format","variable"])

In [ ]:
scaled_df_fama["industry_return"] = scaled_df_fama["value"] / 100

In [ ]:
scaled_df_fama["merafkast"] = scaled_df_fama["adj_return"] - scaled_df_fama["industry_return"]

In [ ]:
scaled_df_fama

In [ ]:
get_na_columns(scaled_df_fama)

In [ ]:
test = scaled_df_fama.groupby("industry_fama")["adj_return","value"].mean()

In [ ]:
test["adj_return"] = test["adj_return"]*100

In [ ]:
test["forskel"] = (test["adj_return"]) - test["value"]

In [ ]:
test

In [ ]:
test.forskel.mean()

In [ ]:
scaled_df

In [ ]:
analytiker_data.head(50)

In [ ]:
scaled_df_fama[scaled_df_fama.Instrument =="02376R102"]

In [ ]:
scaled_df_fama.industry_fama.value_counts(normalize=True)

In [ ]:
scaled_df_fama.industry_fama.value_counts(normalize=False)

In [ ]:
scaled_df_fama.Quarter_end_date.dt.year.value_counts().sort_index()

In [ ]:
scaled_df_fama.info()

# Tilføje EPS fra fremtiden - og omdøbe kolonne navne

In [ ]:
scaled_df_fama.drop("SG&A(XSGAQ)",axis=1,inplace=True)

In [ ]:
#Eksportering af fil
scaled_df_fama.to_csv("/content/drive/MyDrive/Cand.fælles/Speciale/data_prep_2Q_est.csv")

In [ ]:
from pandas.tseries.offsets import MonthEnd
actuals["forecast_period_date"] = actuals["Quarter_end_date"] - MonthEnd(3)


In [ ]:
actuals

In [ ]:
scaled_df.info()

In [ ]:
scaled_df_fama[scaled_df_fama.Instrument=="020813101"].head(2)

In [ ]:
data_output = actuals.merge(scaled_df_fama, how="inner", left_on=["forecast_period_date", "Instrument"], right_on=["Calc Date", "Instrument"])

In [ ]:
data_output

# EPS Acutal - X = Den værdi vi ønsker at forecaste
# EPS Actual - Y = Den senest tilgængelige EPS Værdi 

In [ ]:
drop_list= ["Instrument","CFACSHR", "EndOfMonth","FY","FY_end","Fiscal_year_quarter","HQ_country_code(loc)","company_status","consol","cusip","data_year_quarter","count_obs","currency","datafmt","fqtr",
"index","industry_code(sic)","native_currency","popsrc","reporting_frq","ticker","variable","forecast_period_date","Calc Date_x","Quarter_end_date_x","Period End Date"]

In [ ]:
data_output.drop(drop_list,axis=1,inplace=True)

In [ ]:
data_output.info()

In [ ]:
drop_list = ["Period End Date_y","Period End Date_x","Report_Date_Earnings(Rdq)","date_x","date_y","indfmt","perm","value","date_format"]

In [ ]:
data_output.drop(drop_list,axis=1,inplace=True)

In [ ]:
rename_columns = {"Earnings Per Share Reported - Actual_x" : "EPS_actual", "Earnings Per Share Reported - Actual_y": "EPS_lagged","Quarter_end_date_y": "periode_regnskabstal", "Calc Date_y": "Forecasttidspunkt"}
data_output.rename(columns=rename_columns,inplace=True)

In [ ]:
data_output.info()

In [ ]:
data_output.EPS_actual.idxmax()

In [ ]:
data_output.iloc[91238]

In [ ]:
data_output = data_output[data_output.EPS_actual.between(data_output.EPS_actual.quantile(0.01),data_output.EPS_actual.quantile(0.99))]

In [ ]:
data_output.drop_duplicates(subset=["Forecasttidspunkt","GVKEY"],inplace=True)

In [ ]:
data_output[data_output.GVKEY ==1690]

In [ ]:
actuals[actuals.Instrument == "037833100"]

In [ ]:
compustat[compustat.GVKEY ==1690]

In [ ]:
data_output.shape

In [ ]:
data_output[data_output.PRC.isna()]

In [ ]:
# Aktiedata ind igen
aktiedata_altprice = pd.read_csv("/content/drive/MyDrive/Cand.fælles/Speciale/aktie_data_for_merging.csv")

In [ ]:
aktiedata_altprice = aktiedata_altprice[["date","PERMNO","ALTPRC"]]

In [ ]:
aktiedata_altprice["date"] = pd.to_datetime(aktiedata_altprice["date"])

In [ ]:
from pandas.tseries.offsets import MonthEnd
aktiedata_altprice["slutmåned"] = aktiedata_altprice["date"] - MonthEnd(0)

In [ ]:
aktiedata_altprice

In [ ]:
data_output = data_output.merge(aktiedata_altprice,how="left",left_on=["Forecasttidspunkt","PERMNO"],right_on=["slutmåned","PERMNO"])

In [ ]:
data_output[data_output.isna()]

In [ ]:
data_output.to_csv("/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/input.csv.zip",index=False)

In [ ]:
'''data_dif = data_output.copy()
data_dif["differ"] = data_dif["Forecasttidspunkt"] - data_dif["Forecasttidspunkt"].shift(1)
from pandas import Timedelta
list_uncomp= data_dif[data_dif.differ>Timedelta("100 days")].GVKEY.unique().tolist()
data_output[~data_output.GVKEY.isin(list_uncomp)].to_csv("/content/drive/MyDrive/Cand.fælles/Speciale/datarobot_input/complete_ts.csv.zip")

'''